> ### Note on Labs and Assigments:
>
> 🔧 Look for the **wrench emoji** 🔧 — it highlights where you're expected to take action!
>
> These sections are graded and are not optional.
>

# IS 4487 Lab 6: Data Cleaning

## Outline

- Load and inspect a new dataset (Megatelco)
- Fix column names and data types
- Handle missing values
- Remove duplicate rows
- Review and remove outliers
- Reflect on data quality

In this lab, we’ll clean the data to get it ready for transformations and analysis.

We will continue working with this dataset in **Lab 7**, where we will create new features and apply transformations.

<a href="https://colab.research.google.com/github/Stan-Pugsley/is_4487_base/blob/main/Labs/lab_06_data_cleaning.ipynb" target="_parent">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Megatelco Data Dictionary

 DEMOGRAPHIC VARIABLES:
 - College - has the customer attended some college (one, zero)
 - Income - annual income of customer
 - House - estimated price of the customer's home (if applicable)

 USAGE VARIABLES:
 - Data Overage Mb - Average number of megabytes that the customer used in excess of the plan limit (over last 12 months)
 - Data Leftover Mb - Average number of megabytes that the customer use was below the plan limit (over last 12 months)
 - Data Mb Used - Average number of megabytes used per month (over last 12 months)
 - Text Message Count - Average number of texts per month (over last 12 months)
 - Over 15 Minute Calls Per Month - Average number of calls over 15 minutes in duration per month (over last 12 months)
 - Average Call Duration- Average call duration (over last 12 months)

PHONE VARIABLES:
 - Operating System - Current operating system of phone
 - Handset Price - Retail price of the phone used by the customer

ATTITUDINAL VARIABLES:
 - Reported Satisfaction - Survey response to "How satisfied are you with your current phone plan?" (high, med, low)
 - Reported Usage Level - Survey response to "How much do your use your phone?" (high, med, low)
 - Considering Change of Plan - Survey response to "Are you currently planning to change companies when your contract expires?" (high, med, low)

OTHER VARIABLES
 - Leave - Did this customer churn with the last contract expiration? (LEAVE, STAY)
 - ID - Customer identifier

In [1]:
import pandas as pd

url = "https://raw.githubusercontent.com/Stan-Pugsley/is_4487_base/812e9f15c357a5657a2795631fcaa9d9363cb417/DataSets/megatelco_leave_survey_data_cleaning_v2.csv"
df = pd.read_csv(url)

df.head()

,college,income,data_overage_mb,data_leftover_mb,data_mb_used,text_message_count,house,handset_price,over_15mins_calls_per_month,average_call_duration,reported_satisfaction,reported_usage_level,considering_change_of_plan,leave,id,operating_system
0,one,403137.0,70,0.0,6605.0,199,841317,653.0,5.0,8.0,low,low,yes,LEAVE,8183,Android
1,zero,129700.0,67,16.0,6028.0,134,476664,1193.0,5.0,5.0,low,low,yes,LEAVE,12501,IOS
2,zero,69741.0,60,0.0,1482.0,176,810225,1037.0,3.0,8.0,low,low,yes,STAY,7425,IOS
3,one,377572.0,0,22.0,3005.0,184,826967,1161.0,0.0,5.0,low,low,no,LEAVE,13488,IOS
4,zero,382080.0,0,0.0,1794.0,74,951896,1023.0,0.0,14.0,low,low,yes,STAY,11389,IOS


In [2]:
# create a copy of your dataset for use in part 4
copied_df = df.copy(deep=True)

## Part 1: Review Column Names and Structure

Before cleaning, check the structure of the dataset:

- Are column names consistent (lowercase, no spaces)?
- Are there any typos or redundant labels?
- Do the rows and columns appear aligned? (Are all the columns the same size? Are all the rows the same size?)

Why this matters:
Inconsistent or messy column names can break code and make analysis harder to follow.




In [3]:
# Standardize column names: lowercase, no spaces
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")

# Get column info and data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15016 entries, 0 to 15015
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   college                      15016 non-null  object 
 1   income                       15006 non-null  float64
 2   data_overage_mb              15016 non-null  int64  
 3   data_leftover_mb             14916 non-null  float64
 4   data_mb_used                 14916 non-null  float64
 5   text_message_count           15016 non-null  int64  
 6   house                        15016 non-null  int64  
 7   handset_price                14916 non-null  float64
 8   over_15mins_calls_per_month  15013 non-null  float64
 9   average_call_duration        14916 non-null  float64
 10  reported_satisfaction        15016 non-null  object 
 11  reported_usage_level         15016 non-null  object 
 12  considering_change_of_plan   14201 non-null  object 
 13  leave           

## Part 2: Convert Data Types

Before analysis, make sure each column is stored in the correct format. This helps avoid calculation errors, makes plotting smoother, and ensures models interpret the data correctly.

Think about:
- Are numbers accidentally stored as strings?
- Should repeated text values be converted to categories?
- Are "yes"/"no" columns better represented as binary (0/1) or categorical types?

Fixing data types now saves time and avoids issues later in your workflow.




In [4]:
# Check original data types
print("Original dtypes:\n", df.dtypes)

# Convert yes/no to binary categories
df['considering_change_of_plan'] = df['considering_change_of_plan'].map({'yes': 1, 'no': 0}).astype('category')

# Convert categorical text columns
df['college'] = df['college'].astype('category')
df['house'] = df['house'].astype('category')
df['reported_satisfaction'] = df['reported_satisfaction'].astype('category')
df['operating_system'] = df['operating_system'].astype('category')

# Convert numeric-looking columns from object to float
df['income'] = pd.to_numeric(df['income'], errors='coerce')
df['data_overage_mb'] = pd.to_numeric(df['data_overage_mb'], errors='coerce')
df['data_leftover_mb'] = pd.to_numeric(df['data_leftover_mb'], errors='coerce')
df['data_mb_used'] = pd.to_numeric(df['data_mb_used'], errors='coerce')
df['text_message_count'] = pd.to_numeric(df['text_message_count'], errors='coerce')
df['over_15mins_calls_per_month'] = pd.to_numeric(df['over_15mins_calls_per_month'], errors='coerce')
df['average_call_duration'] = pd.to_numeric(df['average_call_duration'], errors='coerce')

# Check updated data types
print("\nUpdated dtypes:\n", df.dtypes)


Original dtypes:
 college                         object
income                         float64
data_overage_mb                  int64
data_leftover_mb               float64
data_mb_used                   float64
text_message_count               int64
house                            int64
handset_price                  float64
over_15mins_calls_per_month    float64
average_call_duration          float64
reported_satisfaction           object
reported_usage_level            object
considering_change_of_plan      object
leave                           object
id                               int64
operating_system                object
dtype: object

Updated dtypes:
 college                        category
income                          float64
data_overage_mb                   int64
data_leftover_mb                float64
data_mb_used                    float64
text_message_count                int64
house                          category
handset_price                   float64
over_1

### 🔧 Try It Yourself – Part 2

1. Convert the `leave` column from "yes"/"no" to binary (`1`/`0`) and make it a **category**
2. Convert `reported_usage_level` to a **categorical** type
3. Convert `house` to an **integer** type
3. Use `.info()` to confirm the changes


In [9]:
df['leave'] = df['leave'].map({'LEAVE': 1, 'STAY': 0}).astype('category')

df['reported_usage_level'] = df['reported_usage_level'].astype('category')

df['house'] = df['house'].astype(int)

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14986 entries, 0 to 15015
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype   
---  ------                       --------------  -----   
 0   college                      14986 non-null  category
 1   income                       14986 non-null  float64 
 2   data_overage_mb              14986 non-null  int64   
 3   data_leftover_mb             14986 non-null  float64 
 4   data_mb_used                 14986 non-null  float64 
 5   text_message_count           14986 non-null  int64   
 6   house                        14986 non-null  int64   
 7   handset_price                14986 non-null  float64 
 8   over_15mins_calls_per_month  14983 non-null  float64 
 9   average_call_duration        14986 non-null  float64 
 10  reported_satisfaction        14986 non-null  category
 11  reported_usage_level         14986 non-null  category
 12  considering_change_of_plan   14175 non-null  category
 13  leave 

## Part 3: Handle Missing Values

Missing data can break charts, skew stats, and disrupt models — so it needs to be handled carefully.

### Think about:
- Are the missing values random or patterned?
- Can we drop rows, or do we need to fill them?
- Should we use mean, median, or something else?

### Guidelines:
- Drop rows if there are only a few missing and the columns associated with them are essential to keep intact
- Use median to replace outliers in numeric columns
- Use 0 if the missing value means “none” (e.g. If the value was in response to: “do you have a history of chronic illness?” and the value was just left blank, we can assume that that blank just means “none” (the patient has no history of chronic illness))
- Use mode to replace categorical values

Cleaning missing values early avoids bigger problems later.

-----


**Note on `.loc` and Warnings** - When assigning values to a DataFrame, especially after filtering or copying, it's best to use `.loc` to avoid **`SettingWithCopyWarning`**. This ensures that you're updating the original data and not a temporary view of it.


In [5]:
# View missing value counts
print("Missing values per column:\n", df.isnull().sum())

# Fill 'handset_price' with median
df['handset_price'] = df['handset_price'].fillna(df['handset_price'].median())

# Drop rows with missing 'income' (if very few)
df = df.dropna(subset=['income']).copy()

# Fill missing 'data_leftover_mb' with 0 if it logically means no leftover data
df.loc[:, 'data_leftover_mb'] = df['data_leftover_mb'].fillna(0)

# Fill 'average_call_duration' with median if necessary
df.loc[:, 'average_call_duration'] = df['average_call_duration'].fillna(df['average_call_duration'].median())

# Fill 'data_mb_used' with median
df.loc[:, 'data_mb_used'] = df['data_mb_used'].fillna(df['data_mb_used'].median())

# Confirm updated missing values
print("\nMissing values after handling:\n", df.isnull().sum())


Missing values per column:
 college                          0
income                          10
data_overage_mb                  0
data_leftover_mb               100
data_mb_used                   100
text_message_count               0
house                            0
handset_price                  100
over_15mins_calls_per_month      3
average_call_duration          100
reported_satisfaction            0
reported_usage_level             0
considering_change_of_plan     815
leave                            0
id                               0
operating_system                 0
dtype: int64

Missing values after handling:
 college                          0
income                           0
data_overage_mb                  0
data_leftover_mb                 0
data_mb_used                     0
text_message_count               0
house                            0
handset_price                    0
over_15mins_calls_per_month      3
average_call_duration            0
reported_satisfa

### 🔧 Try It Yourself – Part 3


There are still some missing values in:

- `over_15mins_calls_per_month`
- `considering_change_of_plan`

Decide how to handle them based on what makes the most sense:

- Should you fill them with 0, the median, or something else?
- For categories, would a placeholder like "unknown" or the most common value work?
- Or is it better to drop those rows?

1. Write and execute code to handle the missing values in the remaining two columns.
2. Use `df.isnull().sum()` to confirm all missing values are handled.



In [12]:
df['over_15mins_calls_per_month'] = df['over_15mins_calls_per_month'].fillna(
    df['over_15mins_calls_per_month'].median()
)

df['considering_change_of_plan'] = df['considering_change_of_plan'].fillna(
    df['considering_change_of_plan'].mode()[0]
)

df.isnull().sum()

,0
college,0
income,0
data_overage_mb,0
data_leftover_mb,0
data_mb_used,0
text_message_count,0
house,0
handset_price,0
over_15mins_calls_per_month,0
average_call_duration,0


## Part 4: Remove Duplicate Rows

Sometimes the same row appears more than once due to data entry or processing mistakes. It's important to check for and remove these duplicates.

Think about:
- Are there rows that are exactly the same?
- If duplicates exist, should you keep the first one, the last one, or none?

Why this matters:
Duplicate rows can inflate totals, distort statistics, and lead to inaccurate conclusions.


In [6]:
# Check for exact duplicates
print(f"Number of duplicate rows: {df.duplicated().sum()}")

# Remove them, keeping the first occurrence
df = df.drop_duplicates()

# Confirm result
print(f"Remaining rows after removing duplicates: {len(df)}")

Number of duplicate rows: 17
Remaining rows after removing duplicates: 14989


### 🔧 Try It Yourself – Part 4

1. Use `copied_df.duplicated().sum()` to count how many duplicates are in your dataset.
2. Try using `copied_df.drop_duplicates(keep='last')` instead — what changes?
3. Explore whether duplicate rows share the same ID or just values across all columns and comment on your observation.


In [13]:
print("Number of duplicate rows in copied_df:", copied_df.duplicated().sum())

copied_df_last = copied_df.drop_duplicates(keep='last')
print("Rows remaining after removing duplicates (keep='last'):", len(copied_df_last))

print("Unique IDs before:", copied_df['id'].nunique())
print("Unique IDs after (keep='last'):", copied_df_last['id'].nunique())

Number of duplicate rows in copied_df: 17
Rows remaining after removing duplicates (keep='last'): 14999
Unique IDs before: 11560
Unique IDs after (keep='last'): 11560


Comment: There were 17 duplicate rows in the dataset. After removing them (keeping the last occurrence), the number of rows decreased from 15,016 to 14,999. The number of unique IDs remained the same (11,560 before and after), which means the duplicates were exact copies of the same customers. Therefore, removing them improves data quality without losing information.

## Part 5: Identify and Remove Obvious Outliers

Outliers are values that fall far outside the normal range. They can come from data entry mistakes or rare cases.

- Use summary statistics or visual tools (like boxplots) to find them.
- Look for clearly unrealistic values — e.g., negative prices or extremely high data usage.
- Decide how to handle them:
  - Remove if they’re errors.
  - Keep if they’re valid but rare — or cap them if needed.

Outliers can distort averages, stretch visualizations, and mislead models, so it’s important to address them carefully.



In [7]:
# Remove negative or nonsensical values using business rules

# Example: remove rows where 'handset_price' is negative
df = df[df['handset_price'] >= 0]

# Example: remove rows with unusually long call durations
df = df[df['average_call_duration'] < 1000]

# Example: remove rows with extremely high text message counts
df = df[df['text_message_count'] < 1000]

# View shape after outlier filtering
print("Shape after removing obvious outliers:", df.shape)


Shape after removing obvious outliers: (14986, 16)


### 🔧 Try It Yourself – Part 5

1. Use `df.describe()` to look for columns with extreme minimum or maximum values.
2. Set a threshold for what you think is "too high" or "too low" for:
  - `data_mb_used`
  - `over_15mins_calls_per_month`
  - `income`
3. Remove those outliers using boolean filtering like `df = df[df['column'] < threshold]`

In [15]:
print(df[['data_mb_used', 'over_15mins_calls_per_month', 'income']].describe())

df = df[df['data_mb_used'] < 50000]
df = df[df['over_15mins_calls_per_month'] < 200]
df = df[df['income'] < 1000000]

print("Shape after removing outliers:", df.shape)

       data_mb_used  over_15mins_calls_per_month         income
count  14986.000000                 14986.000000   14986.000000
mean    4201.271487                    10.569331  241988.523288
std     2195.555853                     8.401171  109520.756068
min      471.000000                     0.000000   55704.550000
25%     2304.000000                     3.000000  147808.250000
50%     4221.000000                     9.000000  241653.000000
75%     6063.000000                    17.000000  336442.000000
max     7929.150000                    35.000000  427995.950000
Shape after removing outliers: (14986, 16)


## Part 6: Handle Outliers Using Quantiles

Instead of removing outliers, we can limit their impact by capping extreme values — a method known as **Winsorizing**.

### How to Do It:
- Use `.quantile()` to identify the 1st and 99th percentiles (or other thresholds).
- Use `.clip()` to cap values within that range.

This keeps your dataset intact while reducing the influence of extreme values on your analysis or model.



In [8]:
# Calculate 1st and 99th percentiles for income
income_min, income_max = df['income'].quantile([0.01, 0.99])

# Use .loc to avoid SettingWithCopyWarning and ensure assignment modifies the original DataFrame
df.loc[:, 'income'] = df['income'].clip(lower=income_min, upper=income_max)

# Clip 'data_mb_used' to within 1st and 99th percentiles
usage_min, usage_max = df['data_mb_used'].quantile([0.01, 0.99])
df.loc[:, 'data_mb_used'] = df['data_mb_used'].clip(lower=usage_min, upper=usage_max)

# Clip 'average_call_duration' to reduce the effect of extreme outliers
call_min, call_max = df['average_call_duration'].quantile([0.01, 0.99])
df.loc[:, 'average_call_duration'] = df['average_call_duration'].clip(lower=call_min, upper=call_max)



### 🔧 Try It Yourself – Part 6

1. Use `.quantile([0.01, 0.99])` to find the range for:
  - `text_message_count`
  - `over_15mins_calls_per_month`
2. Apply `.clip(lower=..., upper=...)` to reduce the impact of those outliers
3. Compare the `.describe()` output before and after clipping and comment on what you observe


In [16]:
print("Before clipping:\n", df[['text_message_count', 'over_15mins_calls_per_month']].describe())

text_min, text_max = df['text_message_count'].quantile([0.01, 0.99])
calls_min, calls_max = df['over_15mins_calls_per_month'].quantile([0.01, 0.99])

df.loc[:, 'text_message_count'] = df['text_message_count'].clip(lower=text_min, upper=text_max)
df.loc[:, 'over_15mins_calls_per_month'] = df['over_15mins_calls_per_month'].clip(lower=calls_min, upper=calls_max)

print("\nAfter clipping:\n", df[['text_message_count', 'over_15mins_calls_per_month']].describe())

Before clipping:
        text_message_count  over_15mins_calls_per_month
count        14986.000000                 14986.000000
mean           135.626985                    10.569331
std             48.835016                     8.401171
min             52.000000                     0.000000
25%             93.000000                     3.000000
50%            135.000000                     9.000000
75%            178.000000                    17.000000
max            220.000000                    35.000000

After clipping:
        text_message_count  over_15mins_calls_per_month
count        14986.000000                 14986.000000
mean           135.626318                    10.558254
std             48.811500                     8.370427
min             53.000000                     0.000000
25%             93.000000                     3.000000
50%            135.000000                     9.000000
75%            178.000000                    17.000000
max            219.000000    

Comment: After applying winsorization at the 1st and 99th percentiles, the maximum values for text_message_count and over_15mins_calls_per_month decreased (220 to 219 and 35 to 33). The mean and standard deviation also adjusted slightly, showing that extreme outliers were capped while keeping the overall data distribution intact.


## 🔧 Part 7: Reflection (100 words or less per question)

1. Which step fixed the most issues in the dataset?
2. What surprised you about the structure or values?
3. Do you feel this data is now ready for transformation in Lab 7?


1. Which step fixed the most issues in the dataset?
Handling missing values fixed the most issues because several columns had null entries. By filling them with median, mode, or 0, we ensured the dataset was complete and ready for analysis.

2. What surprised you about the structure or values?
I was surprised that the dataset contained exact duplicate rows with the same IDs. This showed that errors can occur even in structured data and must be cleaned carefully.

3. Do you feel this data is now ready for transformation in Lab 7?
Yes, the dataset is now clean. Missing values, duplicates, and outliers have been addressed, making it ready for transformations and feature engineering in Lab 7.




## Export Your Notebook to Submit in Canvas
- Use the instructions from Lab 1

In [17]:
!jupyter nbconvert --to html "lab_06_FajardoPatrick.ipynb"

[NbConvertApp] Converting notebook lab_06_FajardoPatrick.ipynb to html
[NbConvertApp] Writing 344845 bytes to lab_06_FajardoPatrick.html
